### Import dataset

In [46]:
# import dataset with real data
import pandas as pd
import csv

df = pd.read_csv("pfaf_plants_merged.csv")

### Clean dataset

In [47]:
# delete numerical columns, binary columns, and URLs
df = df.drop(columns=["edibility_rating_search", "medicinal_rating_search", "plant_url", "Edibility Rating", "Image URLs", "Medicinal Rating", "Other Uses Rating", "USDA hardiness", "Weed Potential"])

# delete columns with redundant information
df = df.drop(columns=["use_keyword", "Common Name", "Common Names", "Scientific Name", "Summary"])

# delete not useful columns
df = df.drop(columns=["common_name_search", "Care Requirements", "Cultivation Details", "Known Hazards", "Range", "Special Uses"])

### Clean the rich-text  (delete extra spaces and convert to lowercase)
Exclude from lowercase conversion:
- geographical information.

Exclude from the lowercase conversion and the extra spaces deletion:
- scientific nomenclature (scientific name, family).

In [48]:
import re
import spacy

# load a small English model 
nlp = spacy.load("en_core_web_sm")

def clean_text_space_lowercase(text, to_lower=True):
    if pd.isna(text):
        return ""
    text = re.sub(r"\s+", " ", text.strip())  # remove extra spaces
    if to_lower:
        text = text.lower() # convert to lowercase
    return text

df["Edible uses"] = df["Edible Uses"].apply(clean_text_space_lowercase)
df["Medicinal properties"] = df["Medicinal Properties"].apply(clean_text_space_lowercase)
df["Other uses"] = df["Other Uses"].apply(clean_text_space_lowercase)
df["Propagation"] = df["Propagation"].apply(clean_text_space_lowercase)

def clean_text_space(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"\s+", " ", text.strip())  # remove extra spaces
    return text

df["Native area"] = df["Native Range"].apply(clean_text_space)

In [49]:
# delete the columns that are not needed anymore
df = df.drop(columns=["Edible Uses", "Medicinal Properties", "Other Uses", "Native Range"])

In [50]:
# modify name of some columns
df = df.rename(columns={"latin_name_search" : "Scientific name"})

### Chunking the columns with a more structured text

In [51]:
def extract_phrases(text):
    if pd.isna(text):
        return []
    phrases = re.split(r"[;,:.]", text)
    return [phrase.strip().lower() for phrase in phrases if phrase.strip()]

df["Propagation_phrases"] = df["Propagation"].apply(extract_phrases)
df["Edible uses_phrases"] = df["Edible uses"].apply(extract_phrases)
df["Medicinal properties_phrases"] = df["Medicinal properties"].apply(extract_phrases)
df["Other uses_phrases"] = df["Other uses"].apply(extract_phrases)

### Propagation - Germination time vs sowing month (plant's numbers)
- prepare the column for further visualization

In [52]:
# # split the text into phrases based on periods followed by space or end of line
# def split_phrases(text):
#     if pd.isna(text):
#         return []
#     return [phrase.strip().lower() for phrase in re.split(r"[.]\s*", text) if phrase.strip()]

# # define keyword-based categorization
# def categorize_phrase(phrase):
#     if not isinstance(phrase, str):
#         return "unknown"
    
#     phrase = phrase.lower()
#     if any(kw in phrase for kw in ["sow", "sown", "sowing"]):
#         return "sowing"
#     elif any(kw in phrase for kw in ["stratification", "scarify", "pre-soak"]):
#         return "seed preparation"
#     elif any(kw in phrase for kw in ["germinate", "germination"]):
#         return "germination"
#     elif any(kw in phrase for kw in ["prick", "pots", "seedlings"]):
#         return "first potting"
#     elif "division" in phrase:
#         return "division"
#     elif any(kw in phrase for kw in ["plant out", "permanent position", "planted"]):
#         return "first planting"
#     elif any(kw in phrase for kw in ["cutting", "basal", "half-ripe"]):
#         return "cutting"
#     elif any(kw in phrase for kw in ["collect", "pot", "shoots"]):
#         return "second potting"        
#     elif any(kw in phrase for kw in ["root", "ready", "plant"]):
#         return "second planting"
#     else:
#         return "other"

# # apply the function to the Propagation column
# # split and categorize
# df["propagation_steps"] = df["Propagation"].apply(split_phrases)

# # expand and categorize
# df_exploded = df.explode("propagation_steps").copy()
# df_exploded["propagation_phase"] = df_exploded["propagation_steps"].apply(categorize_phrase)

# # group by family and propagation phase
# df_family = (df_exploded.groupby(["Family", "propagation_phase", "propagation_steps"]).size().reset_index(name="count"))

The df_exploded with the code above and the division of df["Propagation"] into df["propagation_steps"] and df["propagation_phase"] could be used into a model.

In [53]:
# # extract the number of weeks/months from the propagation phase sowing
# import numpy as np

# def extract_germination_time(text):
#     if pd.isna(text):
#         return np.nan

#     # Normalize text
#     text = text.lower()

#     # Match expressions like "germinate... X weeks" or "germination... X - Y weeks"
#     pattern = r"(?:germinates|germination(?: takes| is)?)(?:.*?)(\d+)(?:\s*[-to]{1,3}\s*(\d+))?\s*(weeks?|months?)"

#     match = re.search(pattern, text)
#     if match:
#         start = int(match.group(1))
#         end = match.group(2)
#         unit = match.group(3)

#         if end:
#             end = int(end)
#             avg = (start + end) / 2
#         else:
#             avg = start

#         if "month" in unit:
#             avg *= 4  # convert months to weeks

#         return avg  # return average germination time in weeks

#     return np.nan

# # apply the function to the Propagation column
# df["germination_weeks"] = df["Propagation"].apply(extract_germination_time)

In [54]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # --- Function to extract months from a text ---
# def extract_months(text):
#     if pd.isna(text):
#         return []
#     months = [
#         "january", "february", "march", "april", "may", "june",
#         "july", "august", "september", "october", "november", "december"
#     ]
#     found = []
#     for m in months:
#         if re.search(rf"\b{m}\b", str(text).lower()):
#             found.append(m.capitalize())
#     return found

# # --- Function to detect sowing-related propagation steps ---
# def is_sowing_step(text):
#     if pd.isna(text):
#         return False
#     return bool(re.search(r"\bsow|seeding|plant\b", text.lower()))

# # Split propagation column into steps (lowercase and stripped)
# df["propagation_steps"] = df["Propagation"].apply(
#     lambda text: [] if pd.isna(text) else [p.strip().lower() for p in re.split(r"[.]\s*", text) if p.strip()]
# )

# # Keep only sowing-related steps
# df["sowing_steps"] = df["propagation_steps"].apply(
#     lambda steps: [s for s in steps if is_sowing_step(s)]
# )

# # Explode sowing steps for month extraction
# df_exploded = df.explode("sowing_steps").copy()

# # Extract months only from sowing steps
# df_exploded["sowing_months"] = df_exploded["sowing_steps"].apply(extract_months)

# # Explode months and drop missing months or germination weeks
# df_months = df_exploded.explode("sowing_months").dropna(subset=["sowing_months"])
# df_months = df_months.dropna(subset=["germination_weeks"])
# df_months_filtered = df_months.copy()

# # Map month names to numeric x values
# month_order = [
#     "January", "February", "March", "April", "May", "June",
#     "July", "August", "September", "October", "November", "December"
# ]
# month_to_num = {m: i for i, m in enumerate(month_order)}

# df_months_filtered.loc[:, "month_num"] = df_months_filtered["sowing_months"].map(month_to_num)

# # Define germination time bins (y-axis)
# bins = list(range(0, 26, 2))  # [0, 2, 4, ..., 24]
# labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]
# df_months_filtered.loc[:, "germ_bin"] = pd.cut(df_months_filtered["germination_weeks"], bins=bins, labels=labels, right=False)

# # Group by month_num, germ_bin, Family to get counts
# grouped = df_months_filtered.groupby(["month_num", "germ_bin", "Family"], observed=True).size().reset_index(name='count')

# # Pivot to get families as columns
# pivot = grouped.pivot_table(index=["month_num", "germ_bin"], columns="Family", values="count", fill_value=0, observed=True)

# # Calculate total count per bin for circle size
# pivot["total"] = pivot.sum(axis=1)

# # Remove zero totals
# pivot = pivot[pivot["total"] > 0]

# # Prepare plot
# fig, ax = plt.subplots(figsize=(14, 15))

# # Normalize circle sizes
# max_size = 3000  # max circle area
# pivot["size"] = (pivot["total"] / pivot["total"].max()) * max_size

# # Calculate germination bin centers for y-axis
# germ_bin_centers = {label: (bins[i] + bins[i+1]) / 2 for i, label in enumerate(labels)}

# # Colors for families
# unique_families = sorted(df_months_filtered["Family"].unique())
# palette = sns.color_palette("hsv", len(unique_families))
# family_colors = {fam: palette[i] for i, fam in enumerate(unique_families)}

# # Function to draw pie charts at given positions
# def draw_pie(ax, xpos, ypos, sizes, colors, size):
#     start = 0
#     for frac, color in zip(sizes, colors):
#         if frac == 0:
#             continue
#         theta1 = start * 360
#         theta2 = (start + frac) * 360
#         wedge = plt.matplotlib.patches.Wedge(center=(xpos, ypos), r=size,
#                                             theta1=theta1, theta2=theta2,
#                                             facecolor=color, edgecolor='k')
#         ax.add_patch(wedge)
#         start += frac

# # Plot pie charts for each month_num and germ_bin
# for (month_num, germ_bin), row in pivot.iterrows():
#     ypos = germ_bin_centers[germ_bin]
#     xpos = month_num
#     size = np.sqrt(row["size"]) / 100  # adjust scale as needed

#     family_counts = row.drop(["total", "size"])
#     total = family_counts.sum()
#     fracs = family_counts / total if total > 0 else family_counts

#     draw_pie(ax, xpos, ypos, fracs, [family_colors[fam] for fam in family_counts.index], size)

# # Format axes and labels
# ax.set_xticks(range(len(month_order)))
# ax.set_xticklabels(month_order, rotation=45, fontsize=12)
# ax.set_ylabel("Germination Time (weeks)", fontsize=14)
# ax.set_yticks(list(germ_bin_centers.values()))
# ax.set_yticklabels(labels, fontsize=12)
# ax.set_xlim(-0.5, 11.5)
# ax.set_ylim(min(bins), max(bins))
# ax.grid(True, linestyle='--', alpha=0.4)

# # Legend for families below plot
# handles = [
#     plt.Line2D([0], [0], marker='o', color='w', label=fam,
#                markerfacecolor=family_colors[fam], markersize=10)
#     for fam in unique_families
# ]

# ax.legend(handles=handles, title="Families",
#           loc='lower center',
#           bbox_to_anchor=(0, -0.25, 1, 0.3),
#           mode='expand', borderaxespad=0,
#           ncol=8,
#           fontsize='small',
#           handletextpad=0.5,
#           labelspacing=1.0)

# plt.subplots_adjust(bottom=0.4)  # bottom margin for legend
# plt.title("When to propagate a plant and in how many weeks the seed will germinate", fontsize=16, pad=20)
# plt.tight_layout(rect=[0, 0.1, 1, 0.95])
# plt.show()

In [55]:
# # --- Function to extract months from a text ---
# def extract_months(text):
#     if pd.isna(text):
#         return []
#     months = [
#         "january", "february", "march", "april", "may", "june",
#         "july", "august", "september", "october", "november", "december"
#     ]
#     found = []
#     for m in months:
#         if re.search(rf"\b{m}\b", str(text).lower()):
#             found.append(m.capitalize())
#     return found

# # --- Function to detect sowing-related propagation steps ---
# def is_sowing_step(text):
#     if pd.isna(text):
#         return False
#     return bool(re.search(r"\bsow|seeding|plant\b", text.lower()))

# def plot_plant_germination(df, plant_name):
#     # Filter for the plant
#     df_plant = df[df["Scientific name"].str.lower() == plant_name.lower()].copy()
    
#     if df_plant.empty:
#         print(f"No data found for plant: {plant_name}")
#         return
    
#     # Split propagation column into steps (lowercase and stripped)
#     df_plant["propagation_steps"] = df_plant["Propagation"].apply(
#         lambda text: [] if pd.isna(text) else [p.strip().lower() for p in re.split(r"[.]\s*", text) if p.strip()]
#     )
    
#     # Keep only sowing-related steps
#     df_plant["sowing_steps"] = df_plant["propagation_steps"].apply(
#         lambda steps: [s for s in steps if is_sowing_step(s)]
#     )
    
#     # Explode sowing steps for month extraction
#     df_exploded = df_plant.explode("sowing_steps").copy()
    
#     # Extract months only from sowing steps
#     df_exploded["sowing_months"] = df_exploded["sowing_steps"].apply(extract_months)
    
#     # Explode months and drop missing months or germination weeks
#     df_months = df_exploded.explode("sowing_months").dropna(subset=["sowing_months"])
#     df_months = df_months.dropna(subset=["germination_weeks"])
#     df_months_filtered = df_months.copy()
    
#     # Map month names to numeric x values
#     month_order = [
#         "January", "February", "March", "April", "May", "June",
#         "July", "August", "September", "October", "November", "December"
#     ]
#     month_to_num = {m: i for i, m in enumerate(month_order)}
    
#     df_months_filtered.loc[:, "month_num"] = df_months_filtered["sowing_months"].map(month_to_num)
    
#     # Define germination time bins (y-axis)
#     bins = list(range(0, 26, 2))  # [0, 2, 4, ..., 24]
#     labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]
#     df_months_filtered.loc[:, "germ_bin"] = pd.cut(df_months_filtered["germination_weeks"], bins=bins, labels=labels, right=False)
    
#     # Group by month_num, germ_bin, Family to get counts
#     grouped = df_months_filtered.groupby(["month_num", "germ_bin", "Family"], observed=True).size().reset_index(name='count')
    
#     # Pivot to get families as columns
#     pivot = grouped.pivot_table(index=["month_num", "germ_bin"], columns="Family", values="count", fill_value=0, observed=True)
    
#     # Calculate total count per bin for circle size
#     pivot["total"] = pivot.sum(axis=1)
    
#     # Remove zero totals
#     pivot = pivot[pivot["total"] > 0]
    
#     if pivot.empty:
#         print(f"No sowing and germination data available to plot for {plant_name}.")
#         return
    
#     # Prepare plot
#     fig, ax = plt.subplots(figsize=(14, 15))
    
#     # Normalize circle sizes
#     max_size = 3000  # max circle area
#     pivot["size"] = (pivot["total"] / pivot["total"].max()) * max_size
    
#     # Calculate germination bin centers for y-axis
#     germ_bin_centers = {label: (bins[i] + bins[i+1]) / 2 for i, label in enumerate(labels)}
    
#     # Colors for families
#     unique_families = sorted(df_months_filtered["Family"].unique())
#     palette = sns.color_palette("hsv", len(unique_families))
#     family_colors = {fam: palette[i] for i, fam in enumerate(unique_families)}
    
#     # Function to draw pie charts at given positions
#     def draw_pie(ax, xpos, ypos, sizes, colors, size):
#         start = 0
#         for frac, color in zip(sizes, colors):
#             if frac == 0:
#                 continue
#             theta1 = start * 360
#             theta2 = (start + frac) * 360
#             wedge = plt.matplotlib.patches.Wedge(center=(xpos, ypos), r=size,
#                                                 theta1=theta1, theta2=theta2,
#                                                 facecolor=color, edgecolor='k')
#             ax.add_patch(wedge)
#             start += frac
    
#     # Plot pie charts for each month_num and germ_bin
#     for (month_num, germ_bin), row in pivot.iterrows():
#         ypos = germ_bin_centers[germ_bin]
#         xpos = month_num
#         size = np.sqrt(row["size"]) / 100  # adjust scale as needed
    
#         family_counts = row.drop(["total", "size"])
#         total = family_counts.sum()
#         fracs = family_counts / total if total > 0 else family_counts
    
#         draw_pie(ax, xpos, ypos, fracs, [family_colors[fam] for fam in family_counts.index], size)
    
#     # Format axes and labels
#     ax.set_xticks(range(len(month_order)))
#     ax.set_xticklabels(month_order, rotation=45, fontsize=12)
#     ax.set_ylabel("Germination Time (weeks)", fontsize=14)
#     ax.set_yticks(list(germ_bin_centers.values()))
#     ax.set_yticklabels(labels, fontsize=12)
#     ax.set_xlim(-0.5, 11.5)
#     ax.set_ylim(min(bins), max(bins))
#     ax.grid(True, linestyle='--', alpha=0.4)
    
#     # Legend for families below plot
#     handles = [
#         plt.Line2D([0], [0], marker='o', color='w', label=fam,
#                    markerfacecolor=family_colors[fam], markersize=10)
#         for fam in unique_families
#     ]
    
#     ax.legend(handles=handles, title="Families",
#               loc='lower center',
#               bbox_to_anchor=(0, -0.15, 1, 0.3),
#               mode='expand', borderaxespad=0,
#               ncol=8,
#               fontsize='small',
#               handletextpad=0.5,
#               labelspacing=1.0)
    
#     plt.subplots_adjust(bottom=0.1)  # bottom margin for legend
#     plt.title(f"When to propagate {plant_name} and germination time", fontsize=16, pad=20)
#     plt.tight_layout(rect=[0, 0.1, 1, 0.95])
#     plt.show()

# # Usage example:
# plot_plant_germination(df, "Abies spectabilis")

### Plant family vs Native area (and Medicinal property)

In [56]:
# import spacy
# import pycountry
# import pandas as pd
# import re
# import time

# # Load spaCy and disable unnecessary components for speed
# nlp = spacy.load("en_core_web_sm")
# nlp.disable_pipes("tagger", "parser")

# # ISO country names only (no subdivisions)
# all_country_names = sorted({c.name for c in pycountry.countries})
# country_names_lower = {name.lower(): name for name in all_country_names}

# # Precompile regex for country names
# country_patterns = [(name, re.compile(r'\b' + re.escape(name) + r'\b', re.IGNORECASE))
#                     for name in all_country_names]

# # Manual aliases
# manual_aliases = {
#     "Taiwan": "Taiwan",
#     "Vietnam": "Viet Nam",
#     "Laos": "Lao People's Democratic Republic",
#     "People's Republic of China": "China",
#     "PRC": "China",
#     "Korea South": "Korea (Republic of)",
#     "South Korea": "Korea (Republic of)",
#     "Korea North": "Korea (Democratic People's Republic of)",
#     "North Korea": "Korea (Democratic People's Republic of)",
#     "Iran": "Iran (Islamic Republic of)",
#     "Turkey": "Türkiye",
#     "Moldova": "Moldova, Republic of",
#     "Yugoslavia": "Yugoslavia",
#     "Czech Republic": "Czechia",
#     "Bolivia" : "Bolivia (Plurinational State of)",
#     "Venezuela" : "Venezuela (Bolivarian Republic of)",
#     "Ivory Coast" : "Côte d'Ivoire",
#     "Côte d'Ivoire" : "Côte d'Ivoire",
#     "Congo" : "Republic of the Congo",
#     "Republic of the Congo": "Republic of the Congo",
#     "Democratic Republic of the Congo": "Congo, Democratic Republic of the",
#     "Tanzania" : "Tanzania, United Republic of",
#     "Syria" : "Syrian Arab Republic",
#     "Brunei" : "Brunei Darussalam"
# }

# # GPEs to exclude (non-countries)
# excluded_gpes = {"Indochina", "Northern Territory", "Himalayas", "Siberia", "Greenland"}

# def clean_text(text):
#     if not isinstance(text, str):
#         return ""

#     # Insert comma before uppercase region labels if missing
#     # This handles cases like 'Turkey TROPICAL ASIA' → 'Turkey, TROPICAL ASIA'
#     text = re.sub(r'(?<![,])(\s+)([A-Z]{2,}(?:\s[A-Z]{2,})*)\b', r', \2', text)

#     # Remove parentheses and their content
#     text = re.sub(r"\(.*?\)", "", text)

#     # Remove punctuation except commas
#     text = re.sub(r"[^\w\s,]", "", text)

#     # Collapse multiple spaces and commas into a single comma
#     text = re.sub(r"\s+", " ", text)           # collapse spaces
#     text = re.sub(r",\s*,", ",", text)        # remove duplicate commas
#     text = re.sub(r"\s*,\s*", ",", text)      # remove spaces around commas

#     # Remove leading/trailing commas and spaces
#     text = text.strip(" ,")

#     return text

# def normalize_aliases(name):
#     key = name.replace(",", "").replace(" ", "").lower()
#     for alias, official in manual_aliases.items():
#         alias_key = alias.replace(" ", "").lower()
#         if key == alias_key:
#             return official
#     return name

# texts = df["Native area"].fillna("").astype(str).tolist()

# results = []
# start = time.time()

# for i, text in enumerate(texts):
#     cleaned = clean_text(text)
#     matched = set()

#     # Flags for presence in original/cleaned text
#     has_china = bool(re.search(r'\bChina\b', cleaned, re.IGNORECASE) or
#                      re.search(r'\bPeople\'s Republic of China\b', cleaned, re.IGNORECASE) or
#                      re.search(r'\bPRC\b', cleaned, re.IGNORECASE))
#     has_taiwan = bool(re.search(r'\bTaiwan\b', cleaned, re.IGNORECASE))
#     has_south_korea = bool(re.search(r'\bSouth Korea\b', cleaned, re.IGNORECASE) or
#                            re.search(r'\bKorea South\b', cleaned, re.IGNORECASE))
#     has_north_korea = bool(re.search(r'\bNorth Korea\b', cleaned, re.IGNORECASE) or
#                            re.search(r'\bKorea North\b', cleaned, re.IGNORECASE))

#     # 1. Regex match known countries
#     for name, pattern in country_patterns:
#         if pattern.search(cleaned):
#             matched.add(name)

#     # 2. Manual alias mapping for any alias found in cleaned text
#     for alias, official in manual_aliases.items():
#         if re.search(r'\b' + re.escape(alias) + r'\b', cleaned, re.IGNORECASE):
#             matched.add(official)

#     # 3. Explicitly add countries if present
#     if has_china:
#         matched.add("China")
#     if has_taiwan:
#         matched.add("Taiwan")
#     if has_south_korea:
#         matched.add("South Korea")
#     if has_north_korea:
#         matched.add("North Korea")

#     # 4. NER fallback with blacklist filtering
#     doc = nlp(cleaned)
#     for ent in doc.ents:
#         if ent.label_ == "GPE":
#             ent_text = ent.text.strip()
#             if ent_text not in excluded_gpes:
#                 ent_text_lower = ent_text.lower()
#                 if ent_text_lower in country_names_lower:
#                     matched.add(country_names_lower[ent_text_lower])

#     # 5. Remove substrings if longer names present (e.g., keep Papua New Guinea, drop Papua)
#     final_matches = set()
#     for name in matched:
#         if not any(name != other and name in other for other in matched):
#             final_matches.add(name)

#     results.append(sorted(final_matches))

# df["native_countries"] = results

# print(f"NER country extraction completed in {time.time() - start:.2f} seconds")

In [57]:
# df.to_excel("plants_native_countries.xlsx", index=False)

In [58]:
# import geopandas as gpd
# import cartopy.io.shapereader as shpreader
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.patches import Patch
# import matplotlib.colors as mcolors
# import itertools

# # 1. Alias replacement
# manual_aliases = {
#     "Taiwan": "Taiwan",
#     "Viet Nam": "Vietnam",
#     "Lao People's Democratic Republic": "Laos",
#     "People's Republic of China": "China",
#     "PRC": "China",
#     "Korea (Republic of)": "South Korea",
#     "Korea (Democratic People's Republic of)": "North Korea",
#     "Iran (Islamic Republic of)": "Iran",
#     "Türkiye": "Turkey",
#     "Moldova": "Moldova",
#     "Moldova, Republic of": "Moldova",
#     "Yugoslavia": "Republic of Serbia",
#     "Czech Republic": "Czechia",
#     "Russia": "Russia",
#     "Russian Federation": "Russia",
#     "United States of America": "United States of America",
#     "United States": "United States of America",
#     "Bolivia (Plurinational State of)": "Bolivia",
#     "Venezuela (Bolivarian Republic of)": "Venezuela",
#     "Côte d'Ivoire": "Côte d'Ivoire",
#     "Congo": "Republic of the Congo",
#     "Republic of the Congo": "Republic of the Congo",
#     "Congo, Democratic Republic of the": "Democratic Republic of the Congo",
#     "Tanzania, United Republic of": "Tanzania",
#     "Syrian Arab Republic": "Syria",
#     "Brunei Darussalam": "Brunei"
# }

# # 2. Top 1 family per country
# native = df[["Family", "native_countries"]].explode("native_countries").rename(columns={"native_countries": "country"})
# native["country"] = native["country"].replace(manual_aliases)

# family_counts = (
#     native.groupby(["country", "Family"])
#     .size()
#     .reset_index(name="count")
#     .sort_values(["country", "count"], ascending=[True, False])
# )
# top1_per_country = family_counts.groupby("country").head(1)
# native_top = native.merge(top1_per_country[["country", "Family"]], on=["country", "Family"])

# # 3. Load world map
# shpfilename = shpreader.natural_earth(resolution='110m', category='cultural', name='admin_0_countries')
# world = gpd.read_file(shpfilename)

# # 4. Merge with map
# world_native = world.copy()
# world_native["Family"] = np.nan

# world_native = world_native.merge(
#     native_top.drop_duplicates(subset=["Family", "country"]),
#     left_on="ADMIN",
#     right_on="country",
#     how="left"
# )

# world_native["Family"] = world_native["Family_y"]
# world_native.drop(columns=["country", "Family_x", "Family_y"], inplace=True)

# # 5. Generate pastel-like colors with forest green included
# families = world_native["Family"].dropna().unique()
# num_families = len(families)

# # Use tab20 colormap (20 distinct colors)
# base_colors = plt.get_cmap('tab20').colors

# # Convert colors to hex strings
# base_colors_hex = [mcolors.rgb2hex(c) for c in base_colors]

# # Insert Forest Green (#228B22) manually at start of palette if not present
# forest_green = "#228B22"
# if forest_green not in base_colors_hex:
#     color_palette = [forest_green] + base_colors_hex
# else:
#     color_palette = base_colors_hex

# # Cycle colors to match number of families
# color_cycle = itertools.cycle(color_palette)

# # Assign colors to families
# color_map = {}
# for fam in families:
#     color_map[fam] = next(color_cycle)

# world_native["color"] = world_native["Family"].map(color_map)

# # 6. Legend order by country top family
# ordered_families = []
# for country in sorted(top1_per_country['country'].unique()):
#     fams = top1_per_country[top1_per_country['country'] == country]['Family'].tolist()
#     ordered_families.extend(fams)

# # Deduplicate while preserving order
# seen = set()
# ordered_families = [x for x in ordered_families if not (x in seen or seen.add(x))]

# # Filter legend families to those in color_map to avoid KeyError
# ordered_families = [fam for fam in ordered_families if fam in color_map]

# # 7. Plot
# fig, ax = plt.subplots(figsize=(18, 14))

# world_native[world_native["Family"].notna()].plot(
#     ax=ax,
#     color=world_native[world_native["Family"].notna()]["color"],
#     edgecolor="black"
# )

# world_native[world_native["Family"].isna()].plot(
#     ax=ax,
#     color="lightgray",
#     edgecolor="white",
#     hatch="///",
#     label="No data"
# )

# legend_elements = [Patch(facecolor=color_map[fam], edgecolor='black', label=fam) for fam in ordered_families]
# legend_elements.append(Patch(facecolor="lightgray", hatch="///", edgecolor="white", label="No data"))

# ax.legend(
#     handles=legend_elements,
#     loc="upper center",
#     bbox_to_anchor=(0.5, 0.1),
#     ncol=5,
#     title="Top Plant Family per Country",
#     frameon=True
# )

# ax.set_title("Top Native Plant Family per Country", fontsize=16)
# ax.axis("off")
# plt.tight_layout()
# plt.show()

In [59]:
# import geopandas as gpd
# import cartopy.io.shapereader as shpreader
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.patches import Patch

# # --- Assume manual_aliases, df, and world map loading done as before ---

# # 1. Explode native countries and apply aliases
# native = df[["Family", "native_countries"]].explode("native_countries").rename(columns={"native_countries": "country"})
# native["country"] = native["country"].replace(manual_aliases)

# # 2. Count families per country and get top 2 per country
# family_counts = (
#     native.groupby(["country", "Family"])
#     .size()
#     .reset_index(name="count")
#     .sort_values(["country", "count"], ascending=[True, False])
# )
# top2_per_country = family_counts.groupby("country").head(2)

# # 3. Combine top 2 families into a single label per country
# top2_labels = (
#     top2_per_country.groupby('country')['Family']
#     .apply(lambda fams: ' | '.join(fams))
#     .reset_index()
#     .rename(columns={'Family': 'Family_Combo'})
# )

# # 4. Load world map
# shpfilename = shpreader.natural_earth(resolution='110m', category='cultural', name='admin_0_countries')
# world = gpd.read_file(shpfilename)

# # 5. Merge top 2 family combos to world GeoDataFrame
# world_native = world.copy()
# world_native = world_native.merge(top2_labels, left_on='ADMIN', right_on='country', how='left')

# # 6. Generate very distinct colors automatically
# unique_combos = world_native['Family_Combo'].dropna().unique()
# num_combos = len(unique_combos)

# # Get distinct colors from matplotlib tab20, tab20b, tab20c combined
# def get_distinct_colors(n):
#     cmap1 = plt.get_cmap('tab20').colors      # 20 colors
#     cmap2 = plt.get_cmap('tab20b').colors     # 20 colors
#     cmap3 = plt.get_cmap('tab20c').colors     # 20 colors
#     base_colors = list(cmap1) + list(cmap2) + list(cmap3)
#     if n > len(base_colors):
#         # Repeat colors if more combos than available colors
#         repeats = int(np.ceil(n / len(base_colors)))
#         base_colors = base_colors * repeats
#     return base_colors[:n]

# color_list = get_distinct_colors(num_combos)
# color_map = dict(zip(unique_combos, color_list))
# world_native['color'] = world_native['Family_Combo'].map(color_map)

# # 7. Plotting
# fig, ax = plt.subplots(figsize=(18, 14))

# world_native[world_native['Family_Combo'].notna()].plot(
#     ax=ax,
#     color=world_native[world_native['Family_Combo'].notna()]['color'],
#     edgecolor='black'
# )

# world_native[world_native['Family_Combo'].isna()].plot(
#     ax=ax,
#     color='lightgray',
#     edgecolor='white',
#     hatch='///',
#     label='No data'
# )

# legend_elements = [Patch(facecolor=color_map[combo], edgecolor='black', label=combo) for combo in unique_combos]
# legend_elements.append(Patch(facecolor='lightgray', hatch='///', edgecolor='white', label='No data'))

# ax.legend(
#     handles=legend_elements,
#     loc='upper center',
#     bbox_to_anchor=(0.5, 0.2),
#     ncol=4,
#     #title='Top 2 Native Plant Families per Country (Combined)',
#     frameon=True
# )

# ax.set_title(' ', fontsize=16)
# ax.axis('off')
# plt.tight_layout()
# # Save image
# #plt.savefig("top2_families_map.png", dpi=300, bbox_inches="tight")
# plt.show()


In [60]:
# import pycountry
# import re
# import pandas as pd

# # Prepare known country names
# known_countries = {c.name for c in pycountry.countries}
# known_subdivisions = {s.name for s in pycountry.subdivisions}

# def extract_geographical_scope_fast(text):
#     if not isinstance(text, str):
#         return []
#     found = set()
#     chunks = [re.sub(r"\(.*?\)", "", c).strip() for c in text.split(",")]
#     for chunk in chunks:
#         if chunk in known_countries or chunk in known_subdivisions:
#             found.add(chunk)
#         else:
#             for country in known_countries:
#                 if chunk.lower() in country.lower():
#                     found.add(country)
#                     break
#     return sorted(found)

# # Extract native countries as lists
# df["native_countries"] = df["Native area"].apply(extract_geographical_scope_fast)

# # Explode the dataframe so each row is one (country, medicinal property) pair
# long_df = df[["native_countries", "Medicinal properties_phrases"]].explode("native_countries")
# long_df = long_df.explode("Medicinal properties_phrases")
# long_df = long_df.dropna(subset=["native_countries", "Medicinal properties_phrases"])

# # Group by country and medicinal property and count occurrences
# counts = (
#     long_df.groupby(["native_countries", "Medicinal properties_phrases"])
#     .size()
#     .reset_index(name="count")
# )

# # For each country, find the medicinal property with the highest count
# top_property_per_country = (
#     counts.sort_values(["native_countries", "count"], ascending=[True, False])
#     .groupby("native_countries")
#     .first()
#     .reset_index()
#     .rename(columns={
#         "native_countries": "country",
#         "Medicinal properties_phrases": "top_medicinal_property",
#         "count": "occurrences"
#     })
# )

# print(top_property_per_country)

In [61]:
# # Example: search for the top medicinal property in "Brazil"
# country_name = "Japan"

# result = top_property_per_country[top_property_per_country["country"] == country_name]

# if not result.empty:
#     top_property = result.iloc[0]["top_medicinal_property"]
#     count = result.iloc[0]["occurrences"]
#     print(f"Top medicinal property in {country_name}: {top_property} (Occurrences: {count})")
# else:
#     print(f"No data found for {country_name}")

### Medicinal property vs Edibility

In [62]:
# import pandas as pd
# import spacy
# import re

# # Load spaCy English model
# nlp = spacy.load("en_core_web_sm")

# # --- Define known plant parts and useful edible use keywords ---
# plant_part_keywords = {
#     "bark", "leaf", "leaves", "root", "seed", "shoot", "gum", "pod",
#     "tip", "fruit", "oil", "trunk", "sap", "resin", "flower", "stem"
# }

# known_uses_keywords = {
#     "tea", "drink", "chewing gum", "paste", "chutney", "seasoning", "thickener",
#     "bread", "food", "sauce", "cooked", "dried", "juice", "fermented", "alcohol",
#     "flour", "gum", "spice", "masticatory", "substitute", "sweetener", "emergency food"
# }

# # --- Clean brackets and references robustly ---
# def clean_text_list(text_list):
#     cleaned = []
#     for phrase in text_list:
#         # Remove bracketed numbers like [123]
#         phrase = re.sub(r'\[\d+\]', '', phrase)
#         # Remove fragments like gum[226, food[105, etc.
#         phrase = re.sub(r'\b\w+\[\d*', '', phrase)
#         # Remove any leftover bracket content
#         phrase = re.sub(r'\[.*?\]', '', phrase)
#         phrase = phrase.strip()
#         cleaned.append(phrase)
#     return cleaned

# # --- Improved keyword extractor ---
# def extract_clean_parts_and_uses(text_list):
#     edible_parts = set()
#     edible_uses = set()

#     for phrase in text_list:
#         doc = nlp(phrase.lower())

#         # 1. Extract from noun chunks
#         for chunk in doc.noun_chunks:
#             tokens = [t.lemma_ for t in chunk if not t.is_stop and t.pos_ == "NOUN"]
#             cleaned = " ".join(tokens).strip()

#             if not cleaned or len(cleaned) < 3:
#                 continue  # skip junk or too short

#             if any(t in plant_part_keywords for t in tokens):
#                 edible_parts.update([t for t in tokens if t in plant_part_keywords])
#             elif any(k in cleaned for k in known_uses_keywords):
#                 edible_uses.add(cleaned)

#         # 2. Extract from standalone nouns
#         for token in doc:
#             if token.pos_ == "NOUN" and not token.is_stop:
#                 lemma = token.lemma_
#                 if lemma in plant_part_keywords:
#                     edible_parts.add(lemma)
#                 elif lemma in known_uses_keywords:
#                     edible_uses.add(lemma)

#     return list(edible_parts), list(edible_uses)

# # --- Normalize phrases into broader categories ---
# def normalize_phrases(phrase_list, groups):
#     normalized = []
#     for phrase in phrase_list:
#         matched = False
#         for key, keywords in groups.items():
#             if any(k in phrase for k in keywords):
#                 normalized.append(key)
#                 matched = True
#                 break
#         if not matched:
#             normalized.append(phrase)
#     # Deduplicate
#     return list(set(normalized))

# # --- Define normalization groups ---
# plant_part_groups = {
#     "bark": ["bark"],
#     "leaf": ["leaf", "leaves"],
#     "root": ["root"],
#     "seed": ["seed", "seedpod", "pod"],
#     "shoot": ["shoot", "tip"],
#     "gum": ["gum", "resin", "sap"],
#     "fruit": ["fruit"],
#     "oil": ["oil"],
#     "flower": ["flower"],
#     "stem": ["stem"],
#     "trunk": ["trunk"],
# }

# use_method_groups = {
#     "tea": ["tea", "tea substitute"],
#     "drink": ["drink", "juice"],
#     "gum": ["chewing gum", "masticatory"],
#     "cooked": ["cooked", "boiled", "roasted"],
#     "dried": ["dried", "smoked"],
#     "bread": ["bread", "cereal", "flour"],
#     "seasoning": ["seasoning", "spice", "condiment"],
#     "paste": ["paste", "chutney", "sauce"],
#     "thickener": ["thickener", "binder"],
#     "emergency food": ["emergency food", "famine food"],
#     "fermented": ["fermented", "alcohol"],
#     # Consolidate all food-related phrases into "food"
#     "food": ["food", "baby food", "breakfast food", "energy food", "food dish"]
# }

# # --- Apply to DataFrame ---
# def process_row(row):
#     if isinstance(row, list):
#         cleaned = clean_text_list(row)
#         parts, uses = extract_clean_parts_and_uses(cleaned)
#         return pd.Series([parts, uses])
#     else:
#         return pd.Series([[], []])  # fallback

# df[["edible_parts", "edible_uses"]] = df["Edible uses_phrases"].apply(process_row)
# df["edible_parts_normalized"] = df["edible_parts"].apply(lambda x: normalize_phrases(x, plant_part_groups))
# df["edible_uses_normalized"] = df["edible_uses"].apply(lambda x: normalize_phrases(x, use_method_groups))

In [63]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd
# from collections import defaultdict

# # Create co-occurrence matrix
# def make_co_matrix(row_col_lists):
#     co_matrix = defaultdict(lambda: defaultdict(int))
#     for _, row in df.iterrows():
#         rows = row.get(row_col_lists[0], [])
#         cols = row.get(row_col_lists[1], [])
#         for r in rows:
#             for c in cols:
#                 co_matrix[r][c] += 1
#     return pd.DataFrame(co_matrix).fillna(0).astype(int).T

# # Medicinal property × edible use
# co_df_uses = make_co_matrix(["edible_uses_normalized", "edible_parts_normalized"])

# # Plot clustermap
# cluster = sns.clustermap(
#     co_df_uses,
#     cmap="YlGnBu",
#     figsize=(14, 10),
#     linewidths=0.5,
#     metric="euclidean",  # or try "cosine" or "correlation"
#     standard_scale=1,    # normalize each row
# )

# # Save the figure to a file
# cluster.fig.savefig("clustermap_plant-parts_edible-uses.png", dpi=300, bbox_inches="tight")

# #plt.suptitle("Clustermap: Plant Parts vs Edible Uses", y=1.02)

# plt.show()

In [64]:
# co_df_parts = make_co_matrix(["Medicinal properties_phrases", "edible_parts_normalized"])

# sns.clustermap(
#     co_df_parts,
#     cmap="YlOrBr",
#     figsize=(14, 10),
#     linewidths=0.5,
#     metric="cosine",  # try different metrics for insights
#     standard_scale=1,
# )

# # Save the figure to a file
# cluster.fig.savefig("clustermap_plant-parts_medicinal-properties.png", dpi=300, bbox_inches="tight")

# #plt.suptitle("Clustermap: Medicinal Properties vs Edible Parts", y=1.02)

# plt.show()

In [65]:
# import pandas as pd
# import spacy
# import re
# from collections import defaultdict

# # Load spaCy model
# nlp = spacy.load("en_core_web_sm")

# # --- Define keywords ---
# plant_part_keywords = {
#     "bark", "leaf", "leaves", "root", "seed", "shoot", "gum", "pod",
#     "tip", "fruit", "oil", "trunk", "sap", "resin", "flower", "stem"
# }
# known_uses_keywords = {
#     "tea", "drink", "chewing gum", "paste", "chutney", "seasoning", "thickener",
#     "bread", "food", "sauce", "cooked", "dried", "juice", "fermented", "alcohol",
#     "flour", "gum", "spice", "masticatory", "substitute", "sweetener", "emergency food"
# }

# # --- Normalization groups ---
# plant_part_groups = {
#     "bark": ["bark"],
#     "leaf": ["leaf", "leaves"],
#     "root": ["root"],
#     "seed": ["seed", "seedpod", "pod"],
#     "shoot": ["shoot", "tip"],
#     "gum": ["gum", "resin", "sap", "gum-"],
#     "fruit": ["fruit"],
#     "oil": ["oil"],
#     "flower": ["flower"],
#     "stem": ["stem"],
#     "trunk": ["trunk"],
# }
# use_method_groups = {
#     "tea": ["tea", "tea substitute"],
#     "drink": ["drink", "juice"],
#     "gum": ["chewing gum", "masticatory", "gum-"],
#     "cooked": ["cooked", "boiled", "roasted"],
#     "dried": ["dried", "smoked"],
#     "bread": ["bread", "cereal", "flour"],
#     "seasoning": ["seasoning", "spice", "condiment"],
#     "paste": ["paste", "chutney", "sauce"],
#     "thickener": ["thickener", "binder"],
#     "emergency food": ["emergency food", "famine food"],
#     "fermented": ["fermented", "alcohol"],
#     "sweetener": ["sweetener", "sweetener portion", "sweetener-portion", "sweetener portion."],
#     "food": ["food", "baby food", "breakfast food", "energy food", "food dish"]
# }

# # --- Cleaning helper ---
# def clean_text_list(text_list):
#     cleaned = []
#     for phrase in text_list:
#         phrase = re.sub(r'\[\d+\]', '', phrase)
#         phrase = re.sub(r'\b\w+\[\d*', '', phrase)
#         phrase = re.sub(r'\[.*?\]', '', phrase)
#         phrase = phrase.strip()
#         cleaned.append(phrase)
#     return cleaned

# # --- Extract parts and uses ---
# def extract_clean_parts_and_uses(text_list):
#     edible_parts = set()
#     edible_uses = set()
#     for phrase in text_list:
#         doc = nlp(phrase.lower())

#         for chunk in doc.noun_chunks:
#             tokens = [t.lemma_ for t in chunk if not t.is_stop and t.pos_ == "NOUN"]
#             cleaned = " ".join(tokens).strip()
#             if not cleaned or len(cleaned) < 3:
#                 continue
#             if any(t in plant_part_keywords for t in tokens):
#                 edible_parts.update([t for t in tokens if t in plant_part_keywords])
#             elif any(k in cleaned for k in known_uses_keywords):
#                 edible_uses.add(cleaned)

#         for token in doc:
#             if token.pos_ == "NOUN" and not token.is_stop:
#                 lemma = token.lemma_
#                 if lemma in plant_part_keywords:
#                     edible_parts.add(lemma)
#                 elif lemma in known_uses_keywords:
#                     edible_uses.add(lemma)

#     return list(edible_parts), list(edible_uses)

# # --- Normalization ---
# def normalize_phrases(phrase_list, groups):
#     normalized = []
#     for phrase in phrase_list:
#         phrase = re.sub(r"[-.,;:_\s]+$", "", phrase)
#         phrase = phrase.strip()
#         if not phrase:
#             continue
#         matched = False
#         for key, keywords in groups.items():
#             if any(k in phrase for k in keywords):
#                 normalized.append(key)
#                 matched = True
#                 break
#         if not matched:
#             normalized.append(phrase)
#     return list(set(filter(None, normalized)))

# # --- Process each row of the DataFrame ---
# def process_row(row):
#     if isinstance(row, list):
#         cleaned = clean_text_list(row)
#         parts, uses = extract_clean_parts_and_uses(cleaned)
#         return pd.Series([parts, uses])
#     else:
#         return pd.Series([[], []])

# # Apply to DataFrame
# df[["edible_parts", "edible_uses"]] = df["Edible uses_phrases"].apply(process_row)
# df["edible_parts_normalized"] = df["edible_parts"].apply(lambda x: normalize_phrases(x, plant_part_groups))
# df["edible_uses_normalized"] = df["edible_uses"].apply(lambda x: normalize_phrases(x, use_method_groups))

# # Split nutritional vs other uses
# nutritional_uses = {"food", "bread", "drink", "sweetener"}
# def split_uses(row):
#     if not isinstance(row, list):
#         return pd.Series([[], []])
#     food_related = [u for u in row if u in nutritional_uses]
#     other_uses = [u for u in row if u not in nutritional_uses]
#     return pd.Series([food_related, other_uses])

# df[["edible_uses_nutritional", "edible_uses_other"]] = df["edible_uses_normalized"].apply(split_uses)

# def make_co_matrix(row_col_lists):
#     from collections import defaultdict
#     co_matrix = defaultdict(lambda: defaultdict(int))
#     for _, row in df.iterrows():
#         rows = row.get(row_col_lists[0], [])
#         cols = row.get(row_col_lists[1], [])
#         for r in rows:
#             for c in cols:
#                 co_matrix[r][c] += 1

#     # Convert to DataFrame and clean
#     matrix = pd.DataFrame(co_matrix).fillna(0).astype(int).T

#     # Remove rows/columns with all zeros
#     matrix = matrix.loc[(matrix != 0).any(axis=1), (matrix != 0).any(axis=0)]

#     # Remove empty string labels from rows and columns
#     matrix = matrix.loc[~(matrix.index.str.strip() == ""), :]
#     matrix = matrix.loc[:, ~(matrix.columns.str.strip() == "")]

#     return matrix

In [66]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # --- Normalize co-occurrence matrix ---
# def normalize_matrix(matrix):
#     return matrix.div(matrix.max(axis=1), axis=0).fillna(0)

# # Create normalized versions
# co_matrix_nutritional = make_co_matrix(["edible_uses_nutritional", "edible_parts_normalized"])
# co_matrix_other = make_co_matrix(["edible_uses_other", "edible_parts_normalized"])

# co_matrix_nutritional = normalize_matrix(co_matrix_nutritional)
# co_matrix_other = normalize_matrix(co_matrix_other)

# # --- Plot two clustermaps stacked vertically ---
# sns.set(style="white")
# # Nutritional Uses Clustermap
# g1 = sns.clustermap(
#     co_matrix_nutritional,
#     cmap="YlGnBu",
#     figsize=(12, 1.5),
#     col_cluster=True,
#     row_cluster=True,
#     annot=False,
#     cbar_pos=(0.02, 0.8, 0.02, 0.15)
# )

# g1.fig.savefig("clustermap_nutritional.png")

# plt.setp(g1.ax_heatmap.get_yticklabels(), rotation=0)

# # Other Uses Clustermap
# g2 = sns.clustermap(
#     co_matrix_other,
#     cmap="YlGnBu",
#     figsize=(12, 8),
#     col_cluster=True,
#     row_cluster=True,
#     annot=False,
#     cbar_pos=(0.02, 0.8, 0.02, 0.15)
# )

# g2.fig.savefig("clustermap_other.png")

# plt.show()


### Medicinal properties vs Family

In [67]:
# import plotly.express as px

# # Explode medicinal properties
# df_exp = df.copy()
# df_exp = df_exp.explode("Medicinal properties_phrases")
# df_exp = df_exp.rename(columns={"Medicinal properties_phrases": "Medicinal Property"})

# # Clean the family names
# df_exp["Family"] = df_exp["Family"].str.replace(r" or .*", "", regex=True)

# # Get top 20 most common families
# top_20_props = df_exp["Family"].value_counts().nlargest(20).index

# # Filter to those top 20 families
# df_filtered = df_exp[df_exp["Family"].isin(top_20_props)].copy()

# # Add dummy count column for Plotly sizing
# df_filtered["Count"] = 1

# # Build the sunburst plot
# sunburst_fig = px.sunburst(
#     df_filtered,
#     path=["Medicinal Property", "Family"],
#     values="Count",
#     title="Medicinal Properties vs top 20 Plant Families",
#     width=1000,
#     height=800
# )

# sunburst_fig.show()

In [68]:
import pandas as pd
import plotly.express as px
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

# Prepare full data without filtering top 20
df_exp = df.copy()
df_exp = df_exp.explode("Medicinal properties_phrases")
df_exp = df_exp.rename(columns={"Medicinal properties_phrases": "Medicinal Property"})

# Clean up family names
df_exp["Family"] = df_exp["Family"].str.replace(r" or .*", "", regex=True)

# Drop NaNs just in case
df_exp = df_exp.dropna(subset=["Medicinal Property", "Family"])

# Add dummy count column
df_exp["Count"] = 1

# Build the full sunburst chart
sunburst_all = px.sunburst(
    df_exp,
    path=["Medicinal Property", "Family"],
    values="Count",
    #title="Medicinal Properties vs Plant Families",
    width=1000,
    height=900
)

sunburst_all.show()